In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, Concatenate

In [1]:
import os
import cv2

# Set the path to your dataset folders
malignant_folder = r"C:\Users\lenovo\OneDrive\Documents\Lungs\Malignant cases"
benign_folder = r"C:\Users\lenovo\OneDrive\Documents\Lungs\Bengin cases"
normal_folder = r"C:\Users\lenovo\OneDrive\Documents\Lungs\Normal cases"

# Get a sample image path from each class
malignant_sample = os.path.join(malignant_folder, os.listdir(malignant_folder)[0])
benign_sample = os.path.join(benign_folder, os.listdir(benign_folder)[0])
normal_sample = os.path.join(normal_folder, os.listdir(normal_folder)[0])

# Load a sample image
sample_image = cv2.imread(malignant_sample)

# Get the image shape
input_shape = sample_image.shape

print("Input Shape:", input_shape)


Input Shape: (512, 512, 3)


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

def unet(input_shape):
    # Input layer
    inputs = Input(input_shape)

    # Encoder (downsampling) path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottom (bottleneck) layer
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder (upsampling) path
    up6 = Concatenate()([UpSampling2D(size=(2, 2))(conv5), conv4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Concatenate()([UpSampling2D(size=(2, 2))(conv6), conv3])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Concatenate()([UpSampling2D(size=(2, 2))(conv7), conv2])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Concatenate()([UpSampling2D(size=(2, 2))(conv8), conv1])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model


In [3]:
unet(input_shape)

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the main directory containing the class folders
main_directory = r"C:\Users\lenovo\OneDrive\Documents\Lungs"

# Define the image data generators with data augmentation for training and validation
train_data_generator = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range=0.1, horizontal_flip=True)
val_data_generator = ImageDataGenerator(rescale=1./255)

# Set the batch size and image dimensions
batch_size = 32
image_size = (256, 256)

# Load and augment the training data
train_data = train_data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42
)

# Load the validation data
val_data = val_data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42
)

# Load the testing data (without data augmentation)
test_data = val_data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42
)


Found 2755 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 2755 images belonging to 3 classes.


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the main directory containing the class folders
main_directory = r"C:\Users\lenovo\OneDrive\Documents\Lungs"

# Define the image data generator
data_generator = ImageDataGenerator(rescale=1./255)

# Set the batch size and image dimensions
batch_size = 32
image_size = (256, 256)

# Load the training data
train_data = data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=['Malignant cases', 'Benign cases', 'Normal cases'],
    subset='training',
    seed=42
)

# Load the validation data
val_data = data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=['Malignant cases', 'Benign cases', 'Normal cases'],
    subset='validation',
    seed=42
)

# Load the testing data
test_data = data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=['Malignant cases', 'Benign cases', 'Normal cases'],
    seed=42
)


Found 2515 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 2515 images belonging to 3 classes.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the main directory containing the class folders
main_directory = r"C:\Users\lenovo\OneDrive\Documents\Lungs"

# Define the image data generator
data_generator = ImageDataGenerator(rescale=1./255)

# Set the batch size and image dimensions
batch_size = 32
image_size = (256, 256)

# Load the data
data = data_generator.flow_from_directory(
    directory=main_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    seed=42
)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    data[0][0], data[0][1], test_size=0.2, random_state=42
)

# Create the data generators
train_data = data_generator.flow(train_images, train_labels, batch_size=batch_size, shuffle=True)
test_data = data_generator.flow(test_images, test_labels, batch_size=batch_size, shuffle=False)


Found 2755 images belonging to 3 classes.


In [3]:
train_data

In [4]:
test_data

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model architecture
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate the model
loss, accuracy = model.evaluate(test_data)


Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 1.0985 - accuracy: 0.6400 - val_loss: 0.9542 - val_accuracy: 0.8571
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 1.0094 - accuracy: 0.6000 - val_loss: 0.4428 - val_accuracy: 0.8571
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 0.9332 - accuracy: 0.6000 - val_loss: 0.7134 - val_accuracy: 0.8571
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 0.8275 - accuracy: 0.6000 - val_loss: 0.7859 - val_accuracy: 0.1429
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 0.8491 - accuracy: 0.3600 - val_loss: 0.7209 - val_accuracy: 0.8571
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 0.8260 - accuracy: 0.6000 - val_loss: 0.6061 - val_accuracy: 0.8571
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 0.8049 - accuracy: 0.6000 - val_loss: 0.5060 - val_accuracy: 0.8571
Epoch 8/10
1/1 [=========================

In [6]:
loss,accuracy

(0.6317917704582214, 0.8571428656578064)

In [7]:
loss, accuracy = model.evaluate(test_data)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

1/1 [==============================] - 0s 209ms/step - loss: 0.6318 - accuracy: 0.8571
Test Loss: 0.6317917704582214
Test Accuracy: 0.8571428656578064


In [8]:
predictions = model.predict(test_data)

1/1 [==============================] - 0s 284ms/step


In [11]:
import tensorflow as tf

model.save('lung_nodule_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('lung_nodule_model.h5')

In [20]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the input image
image = load_img(r"C:\Users\lenovo\OneDrive\Documents\Lungs\Bengin cases\Bengin case (41).jpg", target_size=(256, 256))

# Convert the image to a numpy array
image_array = img_to_array(image)
image_array = image_array / 255.0  # Normalize the image

# Expand dimensions to match the model input shape
image_array = image_array.reshape((1, 256, 256, 3))

In [21]:
prediction = model.predict(image_array)

# Get the predicted class label
class_index = prediction.argmax()
class_label = class_names[class_index]
confidence = prediction[0][class_index]

print('Predicted Class:', class_label)
print('Confidence:', confidence)

1/1 [==============================] - 0s 77ms/step
Predicted Class: Benign cases
Confidence: 0.9996604


In [22]:
# Define the class names in the same order as your training data
class_names = ['Malignant cases', 'Benign cases', 'Normal cases']

# Get the predicted class label
class_index = prediction.argmax()
class_label = class_names[class_index]
confidence = prediction[0][class_index]

print('Predicted Class:', class_label)
print('Confidence:', confidence)

Predicted Class: Benign cases
Confidence: 0.9996604


In [2]:
def convert_to_24hr(time):
    # Separate the time components
    hour, minute, second = map(int, time[:-2].split(':'))
    meridian = time[-2:]

    # Convert to 24-hour format
    if meridian == 'PM' and hour != 12:
        hour += 12
    elif meridian == 'AM' and hour == 12:
        hour = 0

    # Format the time as a string in 24-hour format
    time_24hr = "{:02d}:{:02d}:{:02d}".format(hour, minute, second)

    # Return the converted time
    return time_24hr

# Take input from the user
time_12hr = input("Enter time in 12-hour format (hh:mm:ssAM/PM): ")

# Call the function to convert the time to 24-hour format
time_24hr = convert_to_24hr(time_12hr)

# Print the converted time
print("Time in 24-hour format:", time_24hr)

Enter time in 12-hour format (hh:mm:ssAM/PM): 12:45:56PM
Time in 24-hour format: 12:45:56
